# Домашнее задание №1

In [1]:
import numpy as np
import pandas as pd

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


Recall - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных
Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

bought_list - купленные товары
recommended_list - рекомендованные товары

prices_recommended - цены рекомендованных товаров
prices_bought - цены купленных товаров

Если верить уроку, то в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера, следовательно k=5 это выборка подходящая для Precision, но очень маленькая для Recall. 

In [3]:
def recall_at_k(recommended_list, bought_list, k=50):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    recall = flags.sum() / len(bought_list)

    return recall




def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=50):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    prices_recommended = prices_recommended[:k]
    prices_bought = prices_bought

    flags = np.isin(recommended_list, bought_list)
    money_recall = (flags*prices_recommended).sum() / prices_bought.sum()
     

    return money_recall


## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

**Mean Reciprocal Rank или MRR измеряет, насколько низко в рейтинге находится первый релевантный документ/вещь и т.д. Если MRR близок к 1, это означает, что релевантные результаты находятся в верхней части результатов поиска. Более низкий MRR указывает на более низкое качество поиска, а правильный ответ располагается ниже в результатах поиска.**

In [4]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    # Создаем список рангов
    rankings = np.where(recommended_list == bought_list.reshape(-1, 1))[1] + 1
    
    # Проверяем, если ранги не найдены, возвращаем 0
    if len(rankings) == 0:
        return 0
    
    # Находим минимальный ранг
    min_rank = np.min(rankings)
    
    # Считаем обратный ранг в первом найденном элементе
    reciprocal_rank = 1 / min_rank
    
    # Возвращаем !средний! обратный ранг
    return reciprocal_rank

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [5]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    
    # Создаем список релевантности для рекомендованных элементов
    relevance = np.isin(recommended_list, bought_list).astype(int)
    
    # Создаем список рангов
    rankings = np.where(recommended_list == bought_list.reshape(-1, 1))[1] + 1
    
    # Вычисляем Discounted Cumulative Gain (DCG)
    dcg = np.sum(relevance / np.log2(rankings + 1))
    
    # Сортируем список рекомендованных элементов по убыванию релевантности
    sorted_relevance = np.sort(relevance)[::-1]
    
    # Вычисляем Ideal Discounted Cumulative Gain (IDCG)
    idcg = np.sum(sorted_relevance / np.log2(np.arange(2, len(sorted_relevance) + 2)))
    
    # Вычисляем Normalized Discounted Cumulative Gain (NDCG)
    ndcg = dcg / idcg if idcg > 0 else 0
    
    return ndcg
